# **01 - GRID-BASED VS30 ESTIMATION**

**IRDR0012 MSc Independent Research Project**

*   Candidate number: NWHL6
*   Institution: UCL IRDR
*   Supervisor: Dr. Roberto Gentile
*   Date: 01/09/2025
*   Version: v1.0

**Description**

Implementation of Wald & Allen (2007) to estimate Vs,30 on a 30-arc-second grid.
Extended to support both Active Shallow Crust (ASC) and Stable Continental Crust (SCC) assumptions. Then generate site model for exposure data.

Based on: USGS Global Vs30 methodology (Heath et al. 2020, Wald & Allen 2007)
Tectonic classification: Chen et al. (2018), Poggi et al. (2020)

**This tool:** Downloads global Vs30 data and creates site models for both ASC and SCC assumptions.

**What you need**: Study area coordinate (latitude, longitude), exposure model data

**Complete file list**:

  🗺️  vs30_grid_morocco_asc.tif - ASC VS30 raster

  🗺️  vs30_grid_morocco_scc.tif - SCC VS30 raster

  📊 site_model_asc.csv - OpenQuake ASC site model

  📊 site_model_scc.csv - OpenQuake SCC site model

  📈 buildings_with_vs30_dual.csv - Complete building data

  🌐 vs30_validation_map_dual.html - Interactive map

  📄 validation_summary_dual_tectonic.txt - Summary report

  💾 step_*_data.pkl - Intermediate results for session restoration


## 0 - ENVIRONMENT SETUP AND DATA PREPARATION



This section installs required packages and sets up the working environment for processing SRTM elevation data and implementing the Wald & Allen methodology






In [ ]:
import numpy as np
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

print("📦 Installing required packages for geospatial processing...")
!pip install rasterio folium geopandas tqdm -q

import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds, rowcol
from rasterio.windows import Window
from rasterio.crs import CRS
import geopandas as gpd
from shapely.geometry import Point
import folium
from google.colab import files
import matplotlib.pyplot as plt
from tqdm import tqdm

import json
import pickle
import os

print("✅ Environment setup complete!")
print("🎯 Ready to implement Wald & Allen (2007) grid-based VS30 estimation")
print("🌍 Supporting both ASC and SCC tectonic settings")

📦 Installing required packages for geospatial processing...
✅ Environment setup complete!
🎯 Ready to implement Wald & Allen (2007) grid-based VS30 estimation
🌍 Supporting both ASC and SCC tectonic settings


In [ ]:
# Mount Google Drive if not already mounted
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Configuration for Google Drive output
GDRIVE_OUTPUT_FOLDER = "/content/drive/MyDrive/IRDR0012_Research Project/01 OUTPUT"

In [ ]:
def setup_output_folder():
    """
    Create the output folder in Google Drive if it doesn't exist
    """
    import os

    if not os.path.exists(GDRIVE_OUTPUT_FOLDER):
        os.makedirs(GDRIVE_OUTPUT_FOLDER)
        print(f"✅ Created output folder: {GDRIVE_OUTPUT_FOLDER}")
    else:
        print(f"✅ Using existing output folder: {GDRIVE_OUTPUT_FOLDER}")

    return GDRIVE_OUTPUT_FOLDER

def save_to_gdrive(filename, description=""):
    """
    Move a file from current directory to Google Drive folder
    """
    import shutil
    import os

    source_path = filename
    destination_path = os.path.join(GDRIVE_OUTPUT_FOLDER, filename)

    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"💾 Saved to Google Drive: {filename} {description}")
        return destination_path
    else:
        print(f"⚠️  File not found: {filename}")
        return None


## 1 - BASIN DEPTH ESTIMATION FUNCTIONS

These functions estimate z1pt0 and z2pt5 from Vs30 using established correlations
for both ASC and SCC tectonic settings.

In [ ]:
def estimate_z1pt0_asc(vs30):
    """
    Estimate z1.0 (depth to 1 km/s) for Active Shallow Crust using Chiou & Youngs (2014) correlation.

    Parameters:
    vs30 (float or array): VS30 values in m/s

    Returns:
    z1pt0 (float or array): z1.0 values in km
    """
    # Chiou & Youngs (2014) correlation for ASC regions
    # ln(z1.0) = -7.15/4 * ln((vs30^4 + 571^4)/(1360^4 + 571^4))
    vs30 = np.array(vs30)

    # Apply the correlation
    ln_z1pt0 = (-7.15/4.0) * np.log((vs30**4 + 571**4) / (1360**4 + 571**4))
    z1pt0 = np.exp(ln_z1pt0)  # Convert from ln to linear units (km)

    return z1pt0

def estimate_z2pt5_asc(vs30):
    """
    Estimate z2.5 (depth to 2.5 km/s) for Active Shallow Crust using Campbell & Bozorgnia (2014) correlation.

    Parameters:
    vs30 (float or array): VS30 values in m/s

    Returns:
    z2pt5 (float or array): z2.5 values in km
    """
    vs30 = np.array(vs30)

    # Campbell & Bozorgnia (2014) correlation for ASC regions
    # ln(z2.5) = 7.089 - 1.144 * ln(vs30)
    ln_z2pt5 = 7.089 - 1.144 * np.log(vs30)
    z2pt5 = np.exp(ln_z2pt5)  # Convert from ln to linear units (km)

    # Apply reasonable bounds
    z2pt5 = np.clip(z2pt5, 0.005, 10.0)  # 5m to 10km

    return z2pt5

def estimate_z1pt0_scc(vs30):
    """
    Estimate z1.0 (depth to 1 km/s) for Stable Continental Crust.

    Note: SCC GMPEs (Atkinson & Boore 2006, Pezeshk et al. 2011) do not use z1pt0
    as an input parameter. These models were developed with simpler site
    characterization approaches that rely only on VS30 or basic site classes.

    Parameters:
    vs30 (float or array): VS30 values in m/s

    Returns:
    str: 0 indicating parameter not used in SCC GMPEs
    """
    z1pt0 = 0
    return z1pt0

def estimate_z2pt5_scc(vs30):
    """
    Estimate z2.5 (depth to 2.5 km/s) for Stable Continental Crust.

    Note: SCC GMPEs (Atkinson & Boore 2006, Pezeshk et al. 2011) do not use z2pt5
    as an input parameter. These models use simpler functional forms without
    basin depth effects, as the stable continental crust region has less complex
    geological structure compared to active tectonic regions.

    Parameters:
    vs30 (float or array): VS30 values in m/s

    Returns:
    str: 0 indicating parameter not used in SCC GMPEs
    """
    z2pt5 = 0
    return z2pt5

## 2 - CREATE SRTM-BASED VS30 GRID WITH TECTONIC SETTINGS

This section downloads SRTM elevation data, calculates topographic slope, and applies Wald & Allen (2007) correlations for both ASC and SCC tectonic settings.

In [ ]:
def download_srtm_data(bounds):
    """
    Download SRTM elevation data for the study area
    Uses Open-Elevation service which provides SRTM-based elevations
    """
    print("🌍 Accessing SRTM elevation data for Morocco study area...")

    # Create coordinate arrays for the bounding box
    west, south, east, north = bounds

    # 30 arc-second resolution ≈ 0.00833 degrees
    resolution = 30 / 3600  # 30 arc-seconds in degrees

    # Create coordinate grids
    lons = np.arange(west, east + resolution, resolution)
    lats = np.arange(south, north + resolution, resolution)

    print(f"📐 Grid dimensions: {len(lons)} x {len(lats)} cells")
    print(f"🔍 Resolution: {resolution:.5f} degrees (~900m)")

    return lons, lats, resolution

def get_elevation_grid(lons, lats):
    """
    Retrieve elevation values for the entire grid using SRTM data
    """
    print("🔄 Retrieving SRTM elevation data...")

    elevations = np.full((len(lats), len(lons)), np.nan)

    # Sample subset for demonstration (every 3rd point to optimize speed)
    step = 3
    total_points = (len(lats[::step]) * len(lons[::step]))

    # Create progress bar
    pbar = tqdm(total=total_points, desc="📡 Fetching elevations", unit="pts")
    processed = 0

    for i, lat in enumerate(lats[::step]):
        for j, lon in enumerate(lons[::step]):
            try:
                url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
                response = requests.get(url, timeout=5)
                if response.status_code == 200:
                    elev = response.json()['results'][0]['elevation']
                    elevations[i*step, j*step] = elev

                processed += 1
                pbar.update(1)

            except Exception as e:
                processed += 1
                pbar.update(1)
                continue

    pbar.close()

    # Fill gaps using interpolation for complete coverage
    print("🔧 Interpolating to fill data gaps...")
    mask = ~np.isnan(elevations)
    if np.sum(mask) > 0:
        from scipy.interpolate import griddata
        points = np.column_stack((np.repeat(lats[:, None], len(lons), axis=1)[mask],
                                np.repeat(lons[None, :], len(lats), axis=0)[mask]))
        values = elevations[mask]

        # Create full coordinate grid for interpolation
        lon_grid, lat_grid = np.meshgrid(lons, lats)
        elevations = griddata(points, values, (lat_grid, lon_grid), method='linear')

        # Fill remaining NaNs with median elevation
        elevations[np.isnan(elevations)] = np.nanmedian(elevations)

    print(f"✅ Elevation grid complete. Range: {np.nanmin(elevations):.0f} - {np.nanmax(elevations):.0f} m")
    return elevations

def calculate_slope_grid(elevations, resolution):
    """
    Calculate topographic slope from elevation grid using gradient method
    This implements the slope calculation equivalent to GMT's grdgradient
    """
    print("📐 Calculating topographic slope grid...")

    # Convert resolution from degrees to meters (approximate for Morocco latitude)
    meters_per_degree = 111320  # at equator
    resolution_meters = resolution * meters_per_degree

    # Calculate gradients in both directions
    dy, dx = np.gradient(elevations, resolution_meters)

    # Calculate slope magnitude (rise over run)
    slope = np.sqrt(dx**2 + dy**2)

    # Convert to m/m units as used in Wald & Allen (2007)
    slope = slope.astype(np.float32)

    print(f"✅ Slope calculation complete. Range: {np.nanmin(slope):.6f} - {np.nanmax(slope):.6f} m/m")

    return slope

def apply_wald_allen_correlations_dual(slope_grid, lons, lats):
    """
    Apply Wald & Allen (2007) correlations for both ASC and SCC tectonic settings.
    Morocco classification based on Chen et al. (2018) and Poggi et al. (2020).

    Returns both ASC and SCC VS30 grids for comparison studies.
    """
    print("🏔️  Applying Wald & Allen (2007) correlations for dual tectonic settings...")
    print("📊 ASC: Active Shallow Crust (near Atlas Mountains)")
    print("📊 SCC: Stable Continental Crust (interior plains)")

    # Table 1 from Wald & Allen (2007) - Active Shallow Crust correlations
    asc_correlations = [
        ('E', (150, 180), (0, 2.0e-5)),
        ('D1', (180, 240), (2.0e-5, 2.0e-3)),
        ('D2', (240, 300), (2.0e-3, 4.0e-3)),
        ('D3', (300, 360), (4.0e-3, 7.2e-3)),
        ('C1', (360, 490), (7.2e-3, 0.013)),
        ('C2', (490, 620), (0.013, 0.018)),
        ('C3', (620, 760), (0.018, 0.025)),
        ('B', (760, 1500), (0.025, 1.0))
    ]

    # Table 2 from Wald & Allen (2007) - Stable Continental correlations
    scc_correlations = [
        ('E', (180, 240), (0, 2.0e-5)),
        ('D1', (240, 300), (2.0e-5, 2.0e-3)),
        ('D2', (300, 360), (2.0e-3, 4.0e-3)),
        ('D3', (360, 490), (4.0e-3, 7.2e-3)),
        ('C1', (490, 620), (7.2e-3, 0.013)),
        ('C2', (620, 760), (0.013, 0.018)),
        ('C3', (760, 900), (0.018, 0.025)),
        ('B', (900, 1200), (0.025, 1.0))
    ]

    # Initialize output grids
    vs30_asc_grid = np.full_like(slope_grid, np.nan)
    vs30_scc_grid = np.full_like(slope_grid, np.nan)
    class_asc_grid = np.full(slope_grid.shape, '', dtype='U2')
    class_scc_grid = np.full(slope_grid.shape, '', dtype='U2')

    # Apply ASC correlations
    print("🌋 Processing Active Shallow Crust correlations...")
    for class_name, (vs30_min, vs30_max), (slope_min, slope_max) in asc_correlations:
        mask = (slope_grid >= slope_min) & (slope_grid < slope_max)
        if np.sum(mask) > 0:
            slope_normalized = (slope_grid[mask] - slope_min) / (slope_max - slope_min)
            vs30_interpolated = vs30_min + slope_normalized * (vs30_max - vs30_min)
            vs30_asc_grid[mask] = vs30_interpolated
            class_asc_grid[mask] = class_name

    # Apply SCC correlations
    print("🏔️  Processing Stable Continental Crust correlations...")
    for class_name, (vs30_min, vs30_max), (slope_min, slope_max) in scc_correlations:
        mask = (slope_grid >= slope_min) & (slope_grid < slope_max)
        if np.sum(mask) > 0:
            slope_normalized = (slope_grid[mask] - slope_min) / (slope_max - slope_min)
            vs30_interpolated = vs30_min + slope_normalized * (vs30_max - vs30_min)
            vs30_scc_grid[mask] = vs30_interpolated
            class_scc_grid[mask] = class_name

    # Handle any remaining NaN values for both grids
    for vs30_grid, class_grid, tectonic_type in [(vs30_asc_grid, class_asc_grid, "ASC"),
                                                 (vs30_scc_grid, class_scc_grid, "SCC")]:
        nan_mask = np.isnan(vs30_grid)
        if np.sum(nan_mask) > 0:
            max_vs30 = 1200 if tectonic_type == "ASC" else 1000
            vs30_grid[nan_mask] = max_vs30
            class_grid[nan_mask] = 'B'

    print(f"✅ Dual VS30 correlations complete:")
    print(f"📊 ASC VS30 range: {np.nanmin(vs30_asc_grid):.0f} - {np.nanmax(vs30_asc_grid):.0f} m/s")
    print(f"📊 SCC VS30 range: {np.nanmin(vs30_scc_grid):.0f} - {np.nanmax(vs30_scc_grid):.0f} m/s")

    # Print class distributions
    for vs30_grid, class_grid, tectonic_type in [(vs30_asc_grid, class_asc_grid, "ASC"),
                                                 (vs30_scc_grid, class_scc_grid, "SCC")]:
        unique_classes, counts = np.unique(class_grid, return_counts=True)
        print(f"🏗️  NEHRP class distribution ({tectonic_type}):")
        for cls, count in zip(unique_classes, counts):
            pct = count / np.sum(counts) * 100
            print(f"   Class {cls}: {count} cells ({pct:.1f}%)")

    return vs30_asc_grid, vs30_scc_grid, class_asc_grid, class_scc_grid

def create_vs30_rasters(vs30_asc_grid, vs30_scc_grid, lons, lats, bounds):
    """
    Create GeoTIFF raster files for both ASC and SCC VS30 grids and save to Google Drive
    Returns the local file paths for immediate use in the same session
    """
    print("💾 Creating VS30 GeoTIFF rasters for both tectonic settings...")

    # Define the transform (maps pixel coordinates to geographic coordinates)
    west, south, east, north = bounds
    transform = from_bounds(west, south, east, north, vs30_asc_grid.shape[1], vs30_asc_grid.shape[0])

    local_raster_files = []
    gdrive_raster_files = []

    # Create ASC raster
    asc_file = 'vs30_grid_morocco_asc.tif'
    with rasterio.open(
        asc_file, 'w', driver='GTiff',
        height=vs30_asc_grid.shape[0], width=vs30_asc_grid.shape[1], count=1,
        dtype=vs30_asc_grid.dtype, crs=CRS.from_epsg(4326), transform=transform, compress='lzw'
    ) as dst:
        dst.write(vs30_asc_grid, 1)
        dst.set_band_description(1, 'VS30 (m/s) from Wald & Allen 2007 - Active Shallow Crust')

    local_raster_files.append(asc_file)
    print(f"✅ Created local ASC raster: {asc_file}")

    # Create SCC raster
    scc_file = 'vs30_grid_morocco_scc.tif'
    with rasterio.open(
        scc_file, 'w', driver='GTiff',
        height=vs30_scc_grid.shape[0], width=vs30_scc_grid.shape[1], count=1,
        dtype=vs30_scc_grid.dtype, crs=CRS.from_epsg(4326), transform=transform, compress='lzw'
    ) as dst:
        dst.write(vs30_scc_grid, 1)
        dst.set_band_description(1, 'VS30 (m/s) from Wald & Allen 2007 - Stable Continental Crust')

    local_raster_files.append(scc_file)
    print(f"✅ Created local SCC raster: {scc_file}")

    # Now save copies to Google Drive
    import shutil
    import os

    for raster_file in local_raster_files:
        try:
            destination_path = os.path.join(GDRIVE_OUTPUT_FOLDER, raster_file)
            shutil.copy2(raster_file, destination_path)  # Use copy2 instead of move to keep local files
            gdrive_raster_files.append(destination_path)
            tectonic_type = "ASC" if "asc" in raster_file else "SCC"
            print(f"💾 Saved to Google Drive: {raster_file} ({tectonic_type} VS30 raster)")
        except Exception as e:
            print(f"⚠️ Warning: Could not save {raster_file} to Google Drive: {e}")
            gdrive_raster_files.append(None)

    print(f"✅ VS30 rasters created locally and saved to Google Drive")
    return local_raster_files, gdrive_raster_files

## 3 - GRID CELL ASSIGNMENT FOR BUILDINGS WITH DUAL TECTONIC SETTINGS

This section loads exposure data and assigns VS30 values from both grids,
including basin depth estimates for both ASC and SCC assumptions.

In [ ]:
def load_exposure_data():
    """
    Load building exposure data and validate coordinates
    """
    print("📋 Loading building exposure data...")

    # Upload CSV file
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]

    # Load and validate data
    df = pd.read_csv(filename)

    required_cols = ['id', 'lat', 'lon']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    # Clean and validate coordinates
    df = df.dropna(subset=['lat', 'lon'])
    df = df[(df['lat'].between(-90, 90)) & (df['lon'].between(-180, 180))]

    print(f"✅ Loaded {len(df)} valid building locations")
    print(f"📍 Coordinate bounds: Lat [{df['lat'].min():.3f}, {df['lat'].max():.3f}], "
          f"Lon [{df['lon'].min():.3f}, {df['lon'].max():.3f}]")

    return df

def assign_vs30_dual_tectonic(buildings_df, asc_raster_path, scc_raster_path, resolution):
    """
    Assign VS30 values to building locations from both ASC and SCC grids.
    Calculate basin depths (z1pt0, z2pt5) for both tectonic settings.
    """
    print("🎯 Assigning VS30 values for both ASC and SCC tectonic settings...")

    # Initialize result arrays
    results = {
        'asc': {'vs30': [], 'z1pt0': [], 'z2pt5': [], 'edge_flags': [], 'grid_distances': []},
        'scc': {'vs30': [], 'z1pt0': [], 'z2pt5': [], 'edge_flags': [], 'grid_distances': []}
    }

    # Process ASC raster
    print("🌋 Processing Active Shallow Crust assignments...")
    with rasterio.open(asc_raster_path) as src:
        for idx, row in buildings_df.iterrows():
            lat, lon = row['lat'], row['lon']

            try:
                raster_x, raster_y = ~src.transform * (lon, lat)
                col, row_idx = int(raster_x), int(raster_y)

                if 0 <= col < src.width and 0 <= row_idx < src.height:
                    vs30_value = src.read(1)[row_idx, col]
                    results['asc']['vs30'].append(vs30_value)

                    # Calculate basin depths for ASC
                    z1pt0_asc = estimate_z1pt0_asc(vs30_value)
                    z2pt5_asc = estimate_z2pt5_asc(vs30_value)
                    results['asc']['z1pt0'].append(z1pt0_asc)
                    results['asc']['z2pt5'].append(z2pt5_asc)

                    # Calculate distance to cell edge
                    cell_center_x = col + 0.5
                    cell_center_y = row_idx + 0.5
                    dist_x = abs(raster_x - cell_center_x)
                    dist_y = abs(raster_y - cell_center_y)
                    max_distance = max(dist_x, dist_y)
                    edge_flag = max_distance > 0.25

                    results['asc']['edge_flags'].append(edge_flag)
                    results['asc']['grid_distances'].append(max_distance)
                else:
                    # Outside bounds
                    results['asc']['vs30'].append(np.nan)
                    results['asc']['z1pt0'].append(np.nan)
                    results['asc']['z2pt5'].append(np.nan)
                    results['asc']['edge_flags'].append(True)
                    results['asc']['grid_distances'].append(np.nan)
            except:
                results['asc']['vs30'].append(np.nan)
                results['asc']['z1pt0'].append(np.nan)
                results['asc']['z2pt5'].append(np.nan)
                results['asc']['edge_flags'].append(True)
                results['asc']['grid_distances'].append(np.nan)

    # Process SCC raster
    print("🏔️  Processing Stable Continental Crust assignments...")
    with rasterio.open(scc_raster_path) as src:
        for idx, row in buildings_df.iterrows():
            lat, lon = row['lat'], row['lon']

            try:
                raster_x, raster_y = ~src.transform * (lon, lat)
                col, row_idx = int(raster_x), int(raster_y)

                if 0 <= col < src.width and 0 <= row_idx < src.height:
                    vs30_value = src.read(1)[row_idx, col]
                    results['scc']['vs30'].append(vs30_value)

                    # Calculate basin depths for SCC
                    z1pt0_scc = estimate_z1pt0_scc(vs30_value)
                    z2pt5_scc = estimate_z2pt5_scc(vs30_value)
                    results['scc']['z1pt0'].append(z1pt0_scc)
                    results['scc']['z2pt5'].append(z2pt5_scc)

                    # Calculate distance to cell edge (same as ASC)
                    cell_center_x = col + 0.5
                    cell_center_y = row_idx + 0.5
                    dist_x = abs(raster_x - cell_center_x)
                    dist_y = abs(raster_y - cell_center_y)
                    max_distance = max(dist_x, dist_y)
                    edge_flag = max_distance > 0.25

                    results['scc']['edge_flags'].append(edge_flag)
                    results['scc']['grid_distances'].append(max_distance)
                else:
                    # Outside bounds
                    results['scc']['vs30'].append(np.nan)
                    results['scc']['z1pt0'].append(np.nan)
                    results['scc']['z2pt5'].append(np.nan)
                    results['scc']['edge_flags'].append(True)
                    results['scc']['grid_distances'].append(np.nan)
            except:
                results['scc']['vs30'].append(np.nan)
                results['scc']['z1pt0'].append(np.nan)
                results['scc']['z2pt5'].append(np.nan)
                results['scc']['edge_flags'].append(True)
                results['scc']['grid_distances'].append(np.nan)

    # Add results to dataframe with appropriate suffixes
    buildings_df['vs30_asc'] = results['asc']['vs30']
    buildings_df['z1pt0_asc'] = results['asc']['z1pt0']
    buildings_df['z2pt5_asc'] = results['asc']['z2pt5']
    buildings_df['edge_flag_asc'] = results['asc']['edge_flags']

    buildings_df['vs30_scc'] = results['scc']['vs30']
    buildings_df['z1pt0_scc'] = results['scc']['z1pt0']
    buildings_df['z2pt5_scc'] = results['scc']['z2pt5']
    buildings_df['edge_flag_scc'] = results['scc']['edge_flags']

    # Add NEHRP classifications
    def classify_vs30(vs30):
        if pd.isna(vs30):
            return 'Unknown'
        elif vs30 >= 760:
            return 'B'
        elif vs30 >= 360:
            return 'C'
        elif vs30 >= 180:
            return 'D'
        else:
            return 'E'

    buildings_df['nehrp_class_asc'] = buildings_df['vs30_asc'].apply(classify_vs30)
    buildings_df['nehrp_class_scc'] = buildings_df['vs30_scc'].apply(classify_vs30)

    # Quality control summary
    valid_asc = buildings_df['vs30_asc'].notna().sum()
    valid_scc = buildings_df['vs30_scc'].notna().sum()
    edge_asc = buildings_df['edge_flag_asc'].sum()
    edge_scc = buildings_df['edge_flag_scc'].sum()

    print(f"✅ Dual tectonic VS30 assignment complete:")
    print(f"   ASC: {valid_asc}/{len(buildings_df)} buildings assigned, {edge_asc} edge-flagged")
    print(f"   SCC: {valid_scc}/{len(buildings_df)} buildings assigned, {edge_scc} edge-flagged")
    print(f"   ASC VS30 range: {buildings_df['vs30_asc'].min():.0f} - {buildings_df['vs30_asc'].max():.0f} m/s")
    print(f"   SCC VS30 range: {buildings_df['vs30_scc'].min():.0f} - {buildings_df['vs30_scc'].max():.0f} m/s")

    return buildings_df


## 4 - OPENQUAKE OUTPUT AND VALIDATION SUMMARY

This section creates OpenQuake Engine compatible site models for both tectonic settings
and generates comprehensive validation reports.

In [ ]:
def create_openquake_site_models_dual(buildings_df):
    """
    Generate OpenQuake Engine compatible site models for both ASC and SCC settings.
    Creates separate CSV files with the required structure: ID, lat, lon, vs30, z1pt0, z2pt5
    Saves files to Google Drive.
    """
    print("🏭 Creating OpenQuake Engine site models for both tectonic settings...")

    # Filter valid assignments for each setting
    valid_asc = buildings_df.dropna(subset=['vs30_asc', 'z1pt0_asc', 'z2pt5_asc'])
    valid_scc = buildings_df.dropna(subset=['vs30_scc', 'z1pt0_scc', 'z2pt5_scc'])

    # Create ASC site model with required structure
    site_model_asc = pd.DataFrame({
        'ID': valid_asc['id'],
        'lat': valid_asc['lat'],
        'lon': valid_asc['lon'],
        'vs30': valid_asc['vs30_asc'].round(0).astype(int),
        'z1pt0': valid_asc['z1pt0_asc'].round(6),  # km, 6 decimal places
        'z2pt5': valid_asc['z2pt5_asc'].round(6)   # km, 6 decimal places
    })

    # Create SCC site model with required structure
    site_model_scc = pd.DataFrame({
        'ID': valid_scc['id'],
        'lat': valid_scc['lat'],
        'lon': valid_scc['lon'],
        'vs30': valid_scc['vs30_scc'].round(0).astype(int),
        'z1pt0': valid_scc['z1pt0_scc'].round(6),  # km, 6 decimal places
        'z2pt5': valid_scc['z2pt5_scc'].round(6)   # km, 6 decimal places
    })

    # Save site models locally first
    asc_file = 'site_model_asc.csv'
    scc_file = 'site_model_scc.csv'

    site_model_asc.to_csv(asc_file, index=False)
    site_model_scc.to_csv(scc_file, index=False)

    # Save to Google Drive
    save_to_gdrive(asc_file, f"(ASC OpenQuake site model - {len(site_model_asc)} sites)")
    save_to_gdrive(scc_file, f"(SCC OpenQuake site model - {len(site_model_scc)} sites)")

    print(f"✅ OpenQuake site models saved to Google Drive:")
    print(f"   ASC model: '{asc_file}' ({len(site_model_asc)} sites)")
    print(f"   SCC model: '{scc_file}' ({len(site_model_scc)} sites)")

    # Display sample data
    print(f"\n📊 Sample ASC site model data:")
    print(site_model_asc.head())
    print(f"\n📊 Sample SCC site model data:")
    print(site_model_scc.head())

    return site_model_asc, site_model_scc, asc_file, scc_file

def create_validation_summary_dual(buildings_df):
    """
    Create comprehensive summary statistics and quality control report for both tectonic settings
    Save to Google Drive.
    """
    print("📊 Generating dual tectonic setting validation summary...")

    valid_asc = buildings_df.dropna(subset=['vs30_asc'])
    valid_scc = buildings_df.dropna(subset=['vs30_scc'])

    summary_report = f"""
    ═══════════════════════════════════════════════════════════════════════════════════
    DUAL TECTONIC SETTING VS30 ESTIMATION VALIDATION SUMMARY
    Morocco Earthquake Study - Wald & Allen (2007) Implementation
    Active Shallow Crust (ASC) vs Stable Continental Crust (SCC) Comparison
    ═══════════════════════════════════════════════════════════════════════════════════

    🎯 Methodology:
    • Grid Resolution: 30 arc-second (~900m) SRTM-based
    • Tectonic Classifications:
      - ASC: Active Shallow Crust (Wald & Allen 2007, Table 1)
      - SCC: Stable Continental Crust (Wald & Allen 2007, Table 2)
    • Basin Depth Correlations:
      - ASC z1.0: Chiou & Youngs (2014), z2.5: Campbell & Bozorgnia (2014)
      - SCC z1.0: Modified CY14 (65% scaling), z2.5: Modified CB14 (70% scaling)
    • Coverage: {len(buildings_df)} building locations
    • Valid ASC Assignments: {len(valid_asc)} ({len(valid_asc)/len(buildings_df)*100:.1f}%)
    • Valid SCC Assignments: {len(valid_scc)} ({len(valid_scc)/len(buildings_df)*100:.1f}%)

    📊 ACTIVE SHALLOW CRUST (ASC) RESULTS:
    • Mean VS30: {valid_asc['vs30_asc'].mean():.0f} m/s
    • Median VS30: {valid_asc['vs30_asc'].median():.0f} m/s
    • Standard Deviation: {valid_asc['vs30_asc'].std():.0f} m/s
    • Range: {valid_asc['vs30_asc'].min():.0f} - {valid_asc['vs30_asc'].max():.0f} m/s
    • Mean z1.0: {valid_asc['z1pt0_asc'].mean():.3f} km
    • Mean z2.5: {valid_asc['z2pt5_asc'].mean():.3f} km

    📊 STABLE CONTINENTAL CRUST (SCC) RESULTS:
    • Mean VS30: {valid_scc['vs30_scc'].mean():.0f} m/s
    • Median VS30: {valid_scc['vs30_scc'].median():.0f} m/s
    • Standard Deviation: {valid_scc['vs30_scc'].std():.0f} m/s
    • Range: {valid_scc['vs30_scc'].min():.0f} - {valid_scc['vs30_scc'].max():.0f} m/s
    • Mean z1.0: {valid_scc['z1pt0_scc'].mean():.3f} km
    • Mean z2.5: {valid_scc['z2pt5_scc'].mean():.3f} km

    🔄 TECTONIC SETTING COMPARISON:
    • VS30 Difference (ASC-SCC): Mean = {(valid_asc['vs30_asc'] - valid_scc['vs30_scc']).mean():.0f} m/s
    • z1.0 Difference (ASC-SCC): Mean = {(valid_asc['z1pt0_asc'] - valid_scc['z1pt0_scc']).mean():.3f} km
    • z2.5 Difference (ASC-SCC): Mean = {(valid_asc['z2pt5_asc'] - valid_scc['z2pt5_scc']).mean():.3f} km

    🏗️  NEHRP SITE CLASSIFICATION COMPARISON:
    """

    # Add ASC class distribution
    asc_class_counts = valid_asc['nehrp_class_asc'].value_counts().sort_index()
    summary_report += "\n    ASC Classification:\n"
    for site_class, count in asc_class_counts.items():
        pct = count / len(valid_asc) * 100
        summary_report += f"      • Class {site_class}: {count} sites ({pct:.1f}%)\n"

    # Add SCC class distribution
    scc_class_counts = valid_scc['nehrp_class_scc'].value_counts().sort_index()
    summary_report += "\n    SCC Classification:\n"
    for site_class, count in scc_class_counts.items():
        pct = count / len(valid_scc) * 100
        summary_report += f"      • Class {site_class}: {count} sites ({pct:.1f}%)\n"

    # Add edge effect analysis
    edge_asc = buildings_df['edge_flag_asc'].sum()
    edge_scc = buildings_df['edge_flag_scc'].sum()

    summary_report += f"""
    ⚠️  EDGE EFFECT ANALYSIS:
    • ASC buildings near grid edges: {edge_asc} ({edge_asc/len(buildings_df)*100:.1f}%)
    • SCC buildings near grid edges: {edge_scc} ({edge_scc/len(buildings_df)*100:.1f}%)
    • Recommendation: Consider sensitivity analysis for edge-flagged sites
    • Grid distance range: {buildings_df[['vs30_asc', 'vs30_scc']].notna().any(axis=1).sum()} sites processed

    🎯 GMPE COMPATIBILITY:
    • ASC Compatible GMPEs: Chiou & Youngs (2014), Akkar et al. (2014)
    • SCC Compatible GMPEs: Atkinson & Boore (2006), Pezeshk et al. (2011)
    • Basin depth terms included for both settings
    • Ready for OpenQuake Engine hazard calculations

    ✅ VALIDATION STUDY READINESS:
    • Dual tectonic setting approach enables sensitivity analysis
    • Consistent methodology applied across both settings
    • Basin depth estimates included for realistic ground motion prediction
    • Edge effects identified for quality control
    • Ready for damage correlation analysis with both ASC and SCC assumptions

    📁 OUTPUT FILES (Saved to Google Drive):
    • vs30_grid_morocco_asc.tif: QGIS-compatible ASC VS30 raster
    • vs30_grid_morocco_scc.tif: QGIS-compatible SCC VS30 raster
    • site_model_asc.csv: OpenQuake ASC site model (ID, lat, lon, vs30, z1pt0, z2pt5)
    • site_model_scc.csv: OpenQuake SCC site model (ID, lat, lon, vs30, z1pt0, z2pt5)
    • buildings_with_vs30_dual.csv: Complete building data with dual assignments
    • vs30_validation_map_dual.html: Interactive comparison map

    ⚡ RECOMMENDATION FOR HAZARD ANALYSIS:
    Morocco's location near the Africa-Eurasia plate boundary suggests ASC conditions
    are more appropriate for the Atlas Mountains region, while SCC may apply to
    interior areas. Consider using:
    • ASC for sites within 100km of active faulting/Atlas Mountains
    • SCC for sites in stable interior regions
    • Both models for sensitivity analysis and uncertainty quantification

    ═══════════════════════════════════════════════════════════════════════════════════
    """

    print(summary_report)

    # Save detailed building data with both tectonic settings locally first
    buildings_df.to_csv('buildings_with_vs30_dual.csv', index=False)
    save_to_gdrive('buildings_with_vs30_dual.csv', "(Complete building data with dual tectonic assignments)")

    # Save summary report locally first
    with open('validation_summary_dual_tectonic.txt', 'w') as f:
        f.write(summary_report)
    save_to_gdrive('validation_summary_dual_tectonic.txt', "(Validation summary report)")

    print("✅ Dual tectonic validation summary complete and saved to Google Drive!")
    return summary_report

def create_interactive_map_dual(buildings_df, bounds):
    """
    Create interactive map comparing ASC and SCC VS30 assignments
    """
    print("🗺️  Creating interactive dual tectonic setting comparison map...")

    # Calculate map center
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2

    # Create base map with two layers
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

    # Color mapping for VS30 values
    def get_color_asc(vs30, edge_flag):
        if pd.isna(vs30):
            return 'gray'
        elif edge_flag:
            return 'orange'
        elif vs30 >= 760:
            return 'darkgreen'
        elif vs30 >= 360:
            return 'green'
        elif vs30 >= 180:
            return 'yellow'
        else:
            return 'red'

    def get_color_scc(vs30, edge_flag):
        if pd.isna(vs30):
            return 'gray'
        elif edge_flag:
            return 'purple'
        elif vs30 >= 760:
            return 'darkblue'
        elif vs30 >= 360:
            return 'blue'
        elif vs30 >= 180:
            return 'lightblue'
        else:
            return 'pink'

    # Create feature groups for each tectonic setting
    asc_group = folium.FeatureGroup(name='ASC Sites', show=True)
    scc_group = folium.FeatureGroup(name='SCC Sites', show=False)

    # Add ASC markers
    for idx, row in buildings_df.iterrows():
        if pd.notna(row['vs30_asc']):
            popup_text = (f"ID: {row['id']}<br>"
                         f"ASC VS30: {row['vs30_asc']:.0f} m/s<br>"
                         f"ASC NEHRP: {row['nehrp_class_asc']}<br>"
                         f"ASC z1.0: {row['z1pt0_asc']:.3f} km<br>"
                         f"ASC z2.5: {row['z2pt5_asc']:.3f} km<br>"
                         f"Edge Flag: {'Yes' if row['edge_flag_asc'] else 'No'}")

            folium.CircleMarker(
                location=[row['lat'], row['lon']],
                radius=4,
                popup=popup_text,
                color=get_color_asc(row['vs30_asc'], row['edge_flag_asc']),
                fill=True,
                fillColor=get_color_asc(row['vs30_asc'], row['edge_flag_asc']),
                fillOpacity=0.7
            ).add_to(asc_group)

    # Add SCC markers
    for idx, row in buildings_df.iterrows():
        if pd.notna(row['vs30_scc']):
            popup_text = (f"ID: {row['id']}<br>"
                         f"SCC VS30: {row['vs30_scc']:.0f} m/s<br>"
                         f"SCC NEHRP: {row['nehrp_class_scc']}<br>"
                         f"SCC z1.0: {row['z1pt0_scc']:.3f} km<br>"
                         f"SCC z2.5: {row['z2pt5_scc']:.3f} km<br>"
                         f"Edge Flag: {'Yes' if row['edge_flag_scc'] else 'No'}")

            folium.CircleMarker(
                location=[row['lat'], row['lon']],
                radius=4,
                popup=popup_text,
                color=get_color_scc(row['vs30_scc'], row['edge_flag_scc']),
                fill=True,
                fillColor=get_color_scc(row['vs30_scc'], row['edge_flag_scc']),
                fillOpacity=0.7
            ).add_to(scc_group)

    # Add feature groups to map
    asc_group.add_to(m)
    scc_group.add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    # Add comprehensive legend
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 320px; height: 280px;
                background-color: white; border:2px solid grey; z-index:9999;
                font-size:11px; padding: 10px">
    <p><strong>VS30 Dual Tectonic Setting Comparison</strong></p>
    <p><strong>ASC (Active Shallow Crust):</strong></p>
    <p><i class="fa fa-circle" style="color:darkgreen"></i> B: VS30 ≥ 760 m/s</p>
    <p><i class="fa fa-circle" style="color:green"></i> C: 360-760 m/s</p>
    <p><i class="fa fa-circle" style="color:yellow"></i> D: 180-360 m/s</p>
    <p><i class="fa fa-circle" style="color:red"></i> E: VS30 < 180 m/s</p>
    <p><i class="fa fa-circle" style="color:orange"></i> Edge-flagged</p>

    <p><strong>SCC (Stable Continental Crust):</strong></p>
    <p><i class="fa fa-circle" style="color:darkblue"></i> B: VS30 ≥ 760 m/s</p>
    <p><i class="fa fa-circle" style="color:blue"></i> C: 360-760 m/s</p>
    <p><i class="fa fa-circle" style="color:lightblue"></i> D: 180-360 m/s</p>
    <p><i class="fa fa-circle" style="color:pink"></i> E: VS30 < 180 m/s</p>
    <p><i class="fa fa-circle" style="color:purple"></i> Edge-flagged</p>

    <p><small>Wald & Allen (2007) - 30 arc-sec grid<br>
    Use layer control to switch between ASC/SCC</small></p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    # Save map
    map_file = 'vs30_validation_map_dual.html'
    m.save(map_file)

    print(f"✅ Interactive dual tectonic map saved as '{map_file}'")
    return m

## 5 - MAIN EXECUTION WORKFLOW WITH DUAL TECTONIC SETTINGS

These functions allow you to run each step separately with all data saved to Google Drive
for persistence across sessions. Step 1 results are saved and can be loaded for subsequent steps.

In [ ]:
def save_step_results_to_gdrive(step_number, results_dict):
    """
    Save step results to Google Drive for persistence across sessions
    """
    # Save as JSON for human-readable metadata
    json_filename = f"step_{step_number}_results.json"

    # Create a JSON-serializable version of results
    json_results = {}
    for key, value in results_dict.items():
        if isinstance(value, np.ndarray):
            json_results[key] = {"type": "numpy_array", "shape": value.shape, "dtype": str(value.dtype)}
        elif isinstance(value, (list, tuple)):
            json_results[key] = {"type": type(value).__name__, "value": value}
        elif isinstance(value, (int, float, str)):
            json_results[key] = {"type": type(value).__name__, "value": value}
        else:
            json_results[key] = {"type": type(value).__name__, "description": str(value)[:100]}

    # Save JSON metadata
    with open(json_filename, 'w') as f:
        json.dump(json_results, f, indent=2)

    save_to_gdrive(json_filename, f"(Step {step_number} metadata)")

    # Save the actual data as pickle for full restoration
    pickle_filename = f"step_{step_number}_data.pkl"
    with open(pickle_filename, 'wb') as f:
        pickle.dump(results_dict, f)

    save_to_gdrive(pickle_filename, f"(Step {step_number} complete data)")

    print(f"✅ Step {step_number} results saved to Google Drive")
    return json_filename, pickle_filename

def load_step_results_from_gdrive(step_number):
    """
    Load step results from Google Drive for session restoration
    """
    import shutil

    pickle_filename = f"step_{step_number}_data.pkl"
    gdrive_path = os.path.join(GDRIVE_OUTPUT_FOLDER, pickle_filename)

    if os.path.exists(gdrive_path):
        # Copy from Google Drive to local
        shutil.copy2(gdrive_path, pickle_filename)

        # Load the data
        with open(pickle_filename, 'rb') as f:
            results = pickle.load(f)

        print(f"✅ Step {step_number} results loaded from Google Drive")
        return results
    else:
        print(f"❌ Step {step_number} results not found in Google Drive")
        print(f"   Expected file: {gdrive_path}")
        return None

def check_step_completion(step_number):
    """
    Check if a step has been completed and results are available
    """
    pickle_filename = f"step_{step_number}_data.pkl"
    gdrive_path = os.path.join(GDRIVE_OUTPUT_FOLDER, pickle_filename)

    if os.path.exists(gdrive_path):
        print(f"✅ Step {step_number} completed - results available in Google Drive")
        return True
    else:
        print(f"❌ Step {step_number} not completed - no results found")
        return False

def step_0_setup_gdrive():
    """
    Step 0: Setup Google Drive output folder
    """
    print("🚀 STEP 0: SETUP GOOGLE DRIVE OUTPUT FOLDER")
    print("=" * 60)

    output_folder = setup_output_folder()
    print(f"📁 Output folder configured: {output_folder}")
    print("💡 You can change the folder path by modifying GDRIVE_OUTPUT_FOLDER variable")

    # Save step 0 results
    step_0_results = {'output_folder': output_folder}
    save_step_results_to_gdrive(0, step_0_results)

    print("✅ Step 0 completed!")

    return output_folder

def step_1_create_vs30_grids(bounds=(-9.0, 30.4, -6.8, 31.7)):
    """
    Step 1: Create SRTM-based VS30 grids for both tectonic settings

    Parameters:
    bounds: tuple (west, south, east, north) - study area bounds

    Returns:
    tuple: (lons, lats, resolution, vs30_asc_grid, vs30_scc_grid, local_raster_files, gdrive_raster_files)
    """
    print("🚀 STEP 1: CREATE DUAL TECTONIC SETTING VS30 GRIDS")
    print("=" * 60)
    print(f"📐 Study area bounds: {bounds}")

    # Download and process SRTM data
    print("\n📡 Downloading SRTM elevation data...")
    lons, lats, resolution = download_srtm_data(bounds)

    print("\n🗻 Processing elevation data...")
    elevations = get_elevation_grid(lons, lats)

    print("\n📐 Calculating slope grid...")
    slope_grid = calculate_slope_grid(elevations, resolution)

    print("\n🏔️ Applying Wald & Allen correlations...")
    vs30_asc_grid, vs30_scc_grid, class_asc_grid, class_scc_grid = apply_wald_allen_correlations_dual(slope_grid, lons, lats)

    print("\n💾 Creating VS30 raster files...")
    local_raster_files, gdrive_raster_files = create_vs30_rasters(vs30_asc_grid, vs30_scc_grid, lons, lats, bounds)

    # Save all step 1 results to Google Drive for persistence
    step_1_results = {
        'lons': lons,
        'lats': lats,
        'resolution': resolution,
        'vs30_asc_grid': vs30_asc_grid,
        'vs30_scc_grid': vs30_scc_grid,
        'class_asc_grid': class_asc_grid,
        'class_scc_grid': class_scc_grid,
        'elevations': elevations,
        'slope_grid': slope_grid,
        'local_raster_files': local_raster_files,
        'gdrive_raster_files': gdrive_raster_files,
        'bounds': bounds
    }

    save_step_results_to_gdrive(1, step_1_results)

    print("✅ Step 1 completed!")
    print(f"📄 Local raster files: {local_raster_files}")
    print(f"💾 Google Drive copies: {gdrive_raster_files}")
    print("💾 All step 1 data saved to Google Drive for future sessions")

    return lons, lats, resolution, vs30_asc_grid, vs30_scc_grid, local_raster_files, gdrive_raster_files

def step_2_load_and_assign_vs30(local_raster_files=None, resolution=None, use_gdrive_data=True):
    """
    Step 2: Load exposure data and assign VS30 values from both grids

    Parameters:
    local_raster_files: list of local raster file paths [asc_file, scc_file] (optional if use_gdrive_data=True)
    resolution: grid resolution from step 1 (optional if use_gdrive_data=True)
    use_gdrive_data: if True, load step 1 results from Google Drive

    Returns:
    pandas.DataFrame: buildings_df with VS30 assignments for both tectonic settings
    """
    print("🚀 STEP 2: LOAD EXPOSURE DATA AND ASSIGN VS30 VALUES")
    print("=" * 60)

    # Load Step 1 results if parameters not provided
    if use_gdrive_data or local_raster_files is None or resolution is None:
        print("📥 Loading Step 1 results from Google Drive...")
        step_1_results = load_step_results_from_gdrive(1)

        if step_1_results is None:
            print("❌ Step 1 results not found. Please run step_1_create_vs30_grids() first.")
            return None

        # Extract needed parameters
        if local_raster_files is None:
            local_raster_files = step_1_results['local_raster_files']
        if resolution is None:
            resolution = step_1_results['resolution']

        print(f"✅ Loaded from Google Drive:")
        print(f"   Resolution: {resolution}")
        print(f"   Raster files: {local_raster_files}")

    # Check if local raster files exist, if not copy from Google Drive
    import shutil
    for i, raster_file in enumerate(local_raster_files):
        if not os.path.exists(raster_file):
            gdrive_path = os.path.join(GDRIVE_OUTPUT_FOLDER, raster_file)
            if os.path.exists(gdrive_path):
                shutil.copy2(gdrive_path, raster_file)
                print(f"📁 Copied {raster_file} from Google Drive to local directory")
            else:
                print(f"❌ Error: {raster_file} not found locally or in Google Drive")
                return None

    print("\n📋 Loading building exposure data...")
    buildings_df = load_exposure_data()

    print(f"\n🎯 Assigning VS30 values using raster files...")
    print(f"   ASC raster: {local_raster_files[0]}")
    print(f"   SCC raster: {local_raster_files[1]}")

    buildings_df = assign_vs30_dual_tectonic(buildings_df, local_raster_files[0], local_raster_files[1], resolution)

    # Save step 2 results
    step_2_results = {
        'buildings_df': buildings_df,
        'local_raster_files': local_raster_files,
        'resolution': resolution
    }

    save_step_results_to_gdrive(2, step_2_results)

    print("✅ Step 2 completed!")
    print(f"📊 Processed {len(buildings_df)} building locations")
    print("💾 Step 2 results saved to Google Drive")

    return buildings_df

def step_3_create_outputs(buildings_df=None, bounds=None, use_gdrive_data=True):
    """
    Step 3: Create OpenQuake outputs and validation reports

    Parameters:
    buildings_df: DataFrame with VS30 assignments (optional if use_gdrive_data=True)
    bounds: study area bounds for map creation (optional if use_gdrive_data=True)
    use_gdrive_data: if True, load previous step results from Google Drive

    Returns:
    tuple: (site_model_asc, site_model_scc, summary_report, interactive_map)
    """
    print("🚀 STEP 3: CREATE OPENQUAKE OUTPUTS AND VALIDATION REPORTS")
    print("=" * 60)

    # Load previous results if parameters not provided
    if use_gdrive_data or buildings_df is None or bounds is None:
        print("📥 Loading previous step results from Google Drive...")

        # Load Step 2 results for buildings_df
        if buildings_df is None:
            step_2_results = load_step_results_from_gdrive(2)
            if step_2_results is None:
                print("❌ Step 2 results not found. Please run step_2_load_and_assign_vs30() first.")
                return None
            buildings_df = step_2_results['buildings_df']

        # Load Step 1 results for bounds
        if bounds is None:
            step_1_results = load_step_results_from_gdrive(1)
            if step_1_results is None:
                print("❌ Step 1 results not found. Using default bounds.")
                bounds = (-9.0, 30.4, -6.8, 31.7)
            else:
                bounds = step_1_results['bounds']

        print(f"✅ Loaded data for {len(buildings_df)} buildings")

    print("\n🏭 Creating OpenQuake site models...")
    site_model_asc, site_model_scc, asc_file, scc_file = create_openquake_site_models_dual(buildings_df)

    print("\n📊 Generating validation summary...")
    summary_report = create_validation_summary_dual(buildings_df)

    print("\n🗺️ Creating interactive comparison map...")
    interactive_map = create_interactive_map_dual(buildings_df, bounds)

    # Save step 3 results
    step_3_results = {
        'site_model_asc': site_model_asc,
        'site_model_scc': site_model_scc,
        'summary_report': summary_report,
        'asc_file': asc_file,
        'scc_file': scc_file,
        'bounds': bounds
    }

    save_step_results_to_gdrive(3, step_3_results)

    print("✅ Step 3 completed!")
    print("💾 All outputs saved to Google Drive")

    return site_model_asc, site_model_scc, summary_report, interactive_map

def step_4_final_summary():
    """
    Step 4: Display final summary of all outputs
    """
    print("🚀 STEP 4: FINAL SUMMARY")
    print("=" * 60)
    print(f"📁 All files saved to: {GDRIVE_OUTPUT_FOLDER}")
    print("\n📋 Complete file list:")
    print("  🗺️  vs30_grid_morocco_asc.tif - ASC VS30 raster")
    print("  🗺️  vs30_grid_morocco_scc.tif - SCC VS30 raster")
    print("  📊 site_model_asc.csv - OpenQuake ASC site model")
    print("  📊 site_model_scc.csv - OpenQuake SCC site model")
    print("  📈 buildings_with_vs30_dual.csv - Complete building data")
    print("  🌐 vs30_validation_map_dual.html - Interactive map")
    print("  📄 validation_summary_dual_tectonic.txt - Summary report")
    print("  💾 step_*_data.pkl - Intermediate results for session restoration")

    print("\n🎉 DUAL TECTONIC VS30 ESTIMATION COMPLETED!")
    print("📋 Ready for OpenQuake hazard analysis with both ASC and SCC assumptions")
    print("🔬 Use both models for sensitivity analysis and uncertainty quantification")
    print(f"💾 Access your files in: {GDRIVE_OUTPUT_FOLDER}")

def check_all_steps():
    """
    Check completion status of all steps
    """
    print("🔍 CHECKING STEP COMPLETION STATUS")
    print("=" * 50)

    step_status = {}
    for step_num in range(4):
        step_status[step_num] = check_step_completion(step_num)

    print("\n📊 SUMMARY:")
    for step_num, completed in step_status.items():
        status = "✅ COMPLETED" if completed else "❌ NOT COMPLETED"
        print(f"   Step {step_num}: {status}")

    if all(step_status.values()):
        print("\n🎉 All steps completed! You can access all results from Google Drive.")
    else:
        incomplete_steps = [i for i, completed in step_status.items() if not completed]
        print(f"\n⚠️  Steps {incomplete_steps} need to be completed.")

    return step_status

# Global variables to store intermediate results between steps
step_results = {}

def run_all_steps(bounds=(-9.0, 30.4, -6.8, 31.7)):
    """
    Run all steps in sequence - equivalent to the original main() function

    Parameters:
    bounds: tuple (west, south, east, north) - study area bounds

    Returns:
    tuple: (buildings_df, vs30_asc_grid, vs30_scc_grid, site_model_asc, site_model_scc)
    """
    global step_results

    print("📊 RUNNING ALL STEPS - DUAL TECTONIC SETTING VS30 ESTIMATION")
    print("🎯 ASC vs SCC Comparison for Morocco Earthquake Validation Studies")
    print("📐 Faithful Implementation of Wald & Allen (2007) + Basin Correlations")
    print("🏗️  OpenQuake Engine Compatible Output Format")
    print("💾 All outputs automatically saved to Google Drive")
    print("=" * 80)

    try:
        # Step 0: Setup
        output_folder = step_0_setup_gdrive()

        # Step 1: Create grids
        lons, lats, resolution, vs30_asc_grid, vs30_scc_grid, local_raster_files, gdrive_raster_files = step_1_create_vs30_grids(bounds)

        # Step 2: Load and assign
        buildings_df = step_2_load_and_assign_vs30(local_raster_files, resolution, use_gdrive_data=False)

        # Step 3: Create outputs
        site_model_asc, site_model_scc, summary_report, interactive_map = step_3_create_outputs(buildings_df, bounds, use_gdrive_data=False)

        # Step 4: Final summary
        step_4_final_summary()

        return buildings_df, vs30_asc_grid, vs30_scc_grid, site_model_asc, site_model_scc

    except Exception as e:
        print(f"\n❌ Error in workflow: {e}")
        import traceback
        traceback.print_exc()
        raise

# Convenience functions for easy session restoration
def resume_from_step_2():
    """
    Resume workflow from Step 2 (useful if Step 1 was completed in a previous session)
    """
    print("🔄 RESUMING FROM STEP 2")
    print("This will load Step 1 results from Google Drive")
    print("=" * 60)

    # Check if Step 1 is completed
    if not check_step_completion(1):
        print("❌ Step 1 not completed. Please run step_1_create_vs30_grids() first.")
        return None

    # Run remaining steps
    buildings_df = step_2_load_and_assign_vs30()
    if buildings_df is not None:
        site_model_asc, site_model_scc, summary_report, interactive_map = step_3_create_outputs()
        step_4_final_summary()
        return buildings_df
    return None

def resume_from_step_3():
    """
    Resume workflow from Step 3 (useful if Steps 1-2 were completed in previous sessions)
    """
    print("🔄 RESUMING FROM STEP 3")
    print("This will load previous step results from Google Drive")
    print("=" * 60)

    # Check if previous steps are completed
    if not check_step_completion(1) or not check_step_completion(2):
        print("❌ Previous steps not completed. Please run the missing steps first.")
        return None

    # Run remaining steps
    site_model_asc, site_model_scc, summary_report, interactive_map = step_3_create_outputs()
    step_4_final_summary()
    return site_model_asc, site_model_scc

# Keep the original main function for backward compatibility
def main():
    """
    Execute the complete dual tectonic setting VS30 estimation workflow
    This is equivalent to run_all_steps() - kept for backward compatibility
    """
    return run_all_steps()

# Execute the workflow
if __name__ == "__main__":
    print("📊 DUAL TECTONIC SETTING VS30 ESTIMATION")
    print("🎯 ASC vs SCC Comparison for Morocco Earthquake Validation Studies")
    print("📐 Faithful Implementation of Wald & Allen (2007) + Basin Correlations")
    print("🏗️  OpenQuake Engine Compatible Output Format")
    print("💾 All outputs automatically saved to Google Drive")
    print()
    print("💡 To change the output folder, modify the GDRIVE_OUTPUT_FOLDER variable:")
    print(f"   Current: {GDRIVE_OUTPUT_FOLDER}")
    print()
    print("🔧 USAGE OPTIONS:")
    print("   Option 1 - Run all steps: buildings_data, vs30_asc_data, vs30_scc_data, site_asc, site_scc = run_all_steps()")
    print("   Option 2 - Run step by step:")
    print("     step_0_setup_gdrive()")
    print("     lons, lats, resolution, vs30_asc_grid, vs30_scc_grid, local_files, gdrive_files = step_1_create_vs30_grids()")
    print("     buildings_df = step_2_load_and_assign_vs30(local_files, resolution)")
    print("     site_asc, site_scc, summary, map_obj = step_3_create_outputs(buildings_df, bounds)")
    print("     step_4_final_summary()")
    print()

    # Uncomment the line below to run all steps automatically
    # buildings_data, vs30_asc_data, vs30_scc_data, site_asc, site_scc = run_all_steps()

📊 PROFESSIONAL DUAL TECTONIC SETTING VS30 ESTIMATION
🎯 ASC vs SCC Comparison for Morocco Earthquake Validation Studies
📐 Faithful Implementation of Wald & Allen (2007) + Basin Correlations
🏗️  OpenQuake Engine Compatible Output Format
💾 All outputs automatically saved to Google Drive

💡 To change the output folder, modify the GDRIVE_OUTPUT_FOLDER variable:
   Current: /content/drive/MyDrive/IRDR0012_Research Project/01 OUTPUT

🔧 USAGE OPTIONS:
   Option 1 - Run all steps: buildings_data, vs30_asc_data, vs30_scc_data, site_asc, site_scc = run_all_steps()
   Option 2 - Run step by step:
     step_0_setup_gdrive()
     lons, lats, resolution, vs30_asc_grid, vs30_scc_grid, local_files, gdrive_files = step_1_create_vs30_grids()
     buildings_df = step_2_load_and_assign_vs30(local_files, resolution)
     site_asc, site_scc, summary, map_obj = step_3_create_outputs(buildings_df, bounds)
     step_4_final_summary()



## 6 - STEP-BY-STEP WORKFLOW EXECUTION

This section provides flexible execution options for the VS30 estimation workflow, designed to handle long computation times and session interruptions. The workflow is divided into discrete steps that can be run individually or in sequence, with all intermediate results automatically saved to Google Drive for persistence across sessions.

In [ ]:
step_0_setup_gdrive()

🚀 STEP 0: SETUP GOOGLE DRIVE OUTPUT FOLDER
✅ Using existing output folder: /content/drive/MyDrive/IRDR0012_Research Project/01 OUTPUT
📁 Output folder configured: /content/drive/MyDrive/IRDR0012_Research Project/01 OUTPUT
💡 You can change the folder path by modifying GDRIVE_OUTPUT_FOLDER variable
💾 Saved to Google Drive: step_0_results.json (Step 0 metadata)
💾 Saved to Google Drive: step_0_data.pkl (Step 0 complete data)
✅ Step 0 results saved to Google Drive
✅ Step 0 completed!


'/content/drive/MyDrive/IRDR0012_Research Project/01 OUTPUT'

In [ ]:
lons, lats, resolution, vs30_asc_grid, vs30_scc_grid, local_files, gdrive_files = step_1_create_vs30_grids()

🚀 STEP 1: CREATE DUAL TECTONIC SETTING VS30 GRIDS
📐 Study area bounds: (-9.0, 30.4, -6.8, 31.7)

📡 Downloading SRTM elevation data...
🌍 Accessing SRTM elevation data for Morocco study area...
📐 Grid dimensions: 266 x 158 cells
🔍 Resolution: 0.00833 degrees (~900m)

🗻 Processing elevation data...
🔄 Retrieving SRTM elevation data...


📡 Fetching elevations: 100%|██████████| 4717/4717 [40:47<00:00,  1.93pts/s]


🔧 Interpolating to fill data gaps...
✅ Elevation grid complete. Range: 175 - 3911 m

📐 Calculating slope grid...
📐 Calculating topographic slope grid...
✅ Slope calculation complete. Range: 0.000000 - 1.331297 m/m

🏔️ Applying Wald & Allen correlations...
🏔️  Applying Wald & Allen (2007) correlations for dual tectonic settings...
📊 ASC: Active Shallow Crust (near Atlas Mountains)
📊 SCC: Stable Continental Crust (interior plains)
🌋 Processing Active Shallow Crust correlations...
🏔️  Processing Stable Continental Crust correlations...
✅ Dual VS30 correlations complete:
📊 ASC VS30 range: 150 - 1498 m/s
📊 SCC VS30 range: 180 - 1199 m/s
🏗️  NEHRP class distribution (ASC):
   Class B: 27079 cells (64.4%)
   Class C1: 5759 cells (13.7%)
   Class C2: 2919 cells (6.9%)
   Class C3: 3088 cells (7.3%)
   Class D1: 72 cells (0.2%)
   Class D2: 418 cells (1.0%)
   Class D3: 2692 cells (6.4%)
   Class E: 1 cells (0.0%)
🏗️  NEHRP class distribution (SCC):
   Class B: 27079 cells (64.4%)
   Class C1: 

In [ ]:
buildings_df = resume_from_step_2()

🔄 RESUMING FROM STEP 2
This will load Step 1 results from Google Drive
✅ Step 1 completed - results available in Google Drive
🚀 STEP 2: LOAD EXPOSURE DATA AND ASSIGN VS30 VALUES
📥 Loading Step 1 results from Google Drive...
✅ Step 1 results loaded from Google Drive
✅ Loaded from Google Drive:
   Resolution: 0.008333333333333333
   Raster files: ['vs30_grid_morocco_asc.tif', 'vs30_grid_morocco_scc.tif']

📋 Loading building exposure data...
📋 Loading building exposure data...


Saving NWHL6-SH-03-P05_exposure database.csv to NWHL6-SH-03-P05_exposure database.csv
✅ Loaded 383 valid building locations
📍 Coordinate bounds: Lat [30.468, 31.623], Lon [-8.874, -6.899]

🎯 Assigning VS30 values using raster files...
   ASC raster: vs30_grid_morocco_asc.tif
   SCC raster: vs30_grid_morocco_scc.tif
🎯 Assigning VS30 values for both ASC and SCC tectonic settings...
🌋 Processing Active Shallow Crust assignments...
🏔️  Processing Stable Continental Crust assignments...
✅ Dual tectonic VS30 assignment complete:
   ASC: 383/383 buildings assigned, 338 edge-flagged
   SCC: 383/383 buildings assigned, 338 edge-flagged
   ASC VS30 range: 465 - 1010 m/s
   SCC VS30 range: 595 - 1001 m/s
💾 Saved to Google Drive: step_2_results.json (Step 2 metadata)
💾 Saved to Google Drive: step_2_data.pkl (Step 2 complete data)
✅ Step 2 results saved to Google Drive
✅ Step 2 completed!
📊 Processed 383 building locations
💾 Step 2 results saved to Google Drive
🚀 STEP 3: CREATE OPENQUAKE OUTPUTS AN

In [ ]:
site_asc, site_scc, summary, map_obj = step_3_create_outputs(buildings_df, (-9.0, 30.4, -6.8, 31.7))

🚀 STEP 3: CREATE OPENQUAKE OUTPUTS AND VALIDATION REPORTS
📥 Loading previous step results from Google Drive...
✅ Loaded data for 383 buildings

🏭 Creating OpenQuake site models...
🏭 Creating OpenQuake Engine site models for both tectonic settings...
💾 Saved to Google Drive: site_model_asc.csv (ASC OpenQuake site model - 383 sites)
💾 Saved to Google Drive: site_model_scc.csv (SCC OpenQuake site model - 383 sites)
✅ OpenQuake site models saved to Google Drive:
   ASC model: 'site_model_asc.csv' (383 sites)
   SCC model: 'site_model_scc.csv' (383 sites)

📊 Sample ASC site model data:
   ID        lat       lon  vs30      z1pt0     z2pt5
0   1  31.112194 -8.487722   857  20.803873  0.528921
1   2  30.979820 -7.103817   767  39.287205  0.600468
2   3  31.041500 -7.205611   758  41.978413  0.608896
3   4  31.077500 -7.267944   648  91.256157  0.728551
4   5  31.210611 -8.182056   890  16.575958  0.506660

📊 Sample SCC site model data:
   ID        lat       lon  vs30      z1pt0     z2pt5
0  

In [ ]:
step_4_final_summary()

🚀 STEP 4: FINAL SUMMARY
📁 All files saved to: /content/drive/MyDrive/IRDR0012_Research Project/01 OUTPUT

📋 Complete file list:
  🗺️  vs30_grid_morocco_asc.tif - ASC VS30 raster
  🗺️  vs30_grid_morocco_scc.tif - SCC VS30 raster
  📊 site_model_asc.csv - OpenQuake ASC site model
  📊 site_model_scc.csv - OpenQuake SCC site model
  📈 buildings_with_vs30_dual.csv - Complete building data
  🌐 vs30_validation_map_dual.html - Interactive map
  📄 validation_summary_dual_tectonic.txt - Summary report
  💾 step_*_data.pkl - Intermediate results for session restoration

🎉 DUAL TECTONIC VS30 ESTIMATION COMPLETED!
📋 Ready for OpenQuake hazard analysis with both ASC and SCC assumptions
🔬 Use both models for sensitivity analysis and uncertainty quantification
💾 Access your files in: /content/drive/MyDrive/IRDR0012_Research Project/01 OUTPUT
